In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os


def extract_and_save_features(dataset, feature_extractor, save_path):
    feature_extractor.eval()  # 确保模型处于评估模式
    all_features = []
    all_labels = []
    
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        features = sample['features']
        label = sample['label']
        
        all_features.append(features)
        all_labels.append(label)
    
    all_features = torch.stack(all_features)
    all_labels = torch.tensor(all_labels)
    
    torch.save({'features': all_features, 'labels': all_labels}, save_path)
    print(f"Features and labels saved to {save_path}")


# 忽略所有警告
warnings.filterwarnings("ignore")
# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Total number of localizers in dataset: 394
Total number of batches: 394
Epoch 0/300
----------


Training: 100%|██████████| 394/394 [00:00<00:00, 676.48it/s]


Loss: 0.7830 Acc: 0.7563
Train Accuracy: 0.7563 F1 Score: 0.6695 Spearman Corr: 0.0346 Specificity: 0.6708 QWK: 0.0195 Mean Metrics: 0.3486


Validation: 100%|██████████| 99/99 [00:00<00:00, 1434.95it/s]


Val Loss: 0.6705 Val Acc: 0.7879
Validation Accuracy: 0.7879 F1 Score: 0.6944 Spearman Corr: nan Specificity: 0.6667 QWK: 0.0000 Mean Metrics: nan
Epoch 1/300
----------


...

Val Loss: 3.1105 Val Acc: 0.7576
Validation Accuracy: 0.7576 F1 Score: 0.7399 Spearman Corr: 0.1084 Specificity: 0.7523 QWK: 0.1130 Mean Metrics: 0.4284
Epoch 300/300
----------


Training: 100%|██████████| 394/394 [00:00<00:00, 786.91it/s]


Loss: 0.0002 Acc: 1.0000
Train Accuracy: 1.0000 F1 Score: 1.0000 Spearman Corr: 1.0000 Specificity: 1.0000 QWK: 1.0000 Mean Metrics: 1.0000


Validation: 100%|██████████| 99/99 [00:00<00:00, 1730.58it/s]

Val Loss: 3.0114 Val Acc: 0.7475
Validation Accuracy: 0.7475 F1 Score: 0.7413 Spearman Corr: 0.1405 Specificity: 0.7602 QWK: 0.1435 Mean Metrics: 0.4464


In [ ]:
1-1

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os


def extract_and_save_features(dataset, feature_extractor, save_path):
    feature_extractor.eval()  # 确保模型处于评估模式
    all_features = []
    all_labels = []
    
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        features = sample['features']
        label = sample['label']
        
        all_features.append(features)
        all_labels.append(label)
    
    all_features = torch.stack(all_features)
    all_labels = torch.tensor(all_labels)
    
    torch.save({'features': all_features, 'labels': all_labels}, save_path)
    print(f"Features and labels saved to {save_path}")


# 忽略所有警告
warnings.filterwarnings("ignore")
# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9495 images.
Processed 101/9495 images.
Processed 201/9495 images.
Processed 301/9495 images.
Processed 401/9495 images.
Processed 501/9495 images.
Processed 601/9495 images.
Processed 701/9495 images.
Processed 801/9495 images.
Processed 901/9495 images.
Processed 1001/9495 images.
Processed 1101/9495 images.
Processed 1201/9495 images.
Processed 1301/9495 images.
Processed 1401/9495 images.
Processed 1501/9495 images.
Processed 1601/9495 images.
Processed 1701/9495 images.
Processed 1801/9495 images.
Processed 1901/9495 images.
Processed 2001/9495 images.
Processed 2101/9495 images.
Processed 2201/9495 images.
Processed 2301/9495 images.
Processed 2401/9495 images.
Processed 2501/9495 images.
Processed 2601/9495 images.
Processed 2701/9495 images.
Processed 2801/9495 images.
Processed 2901/9495 images.
Processed 3001/9495 images.
Processed 3101/9495 images.
Processed 3201/9495 images.
Processed 3301/94

Training: 100%|██████████| 394/394 [00:00<00:00, 708.28it/s]


Loss: 0.6598 Acc: 0.7944
Train Accuracy: 0.7944 F1 Score: 0.7079 Spearman Corr: 0.0059 Specificity: 0.6648 QWK: 0.0016 Mean Metrics: 0.3450


Validation: 100%|██████████| 99/99 [00:00<00:00, 1744.53it/s]


Val Loss: 0.6170 Val Acc: 0.8182
Validation Accuracy: 0.8182 F1 Score: 0.7364 Spearman Corr: nan Specificity: 0.6667 QWK: 0.0000 Mean Metrics: nan
Epoch 1/300
----------


...

Val Loss: 2.5125 Val Acc: 0.7475
Validation Accuracy: 0.7475 F1 Score: 0.7363 Spearman Corr: -0.1264 Specificity: 0.7295 QWK: -0.1295 Mean Metrics: 0.3025
Epoch 300/300
----------


Training: 100%|██████████| 394/394 [00:00<00:00, 399.80it/s]


Loss: 0.0010 Acc: 1.0000
Train Accuracy: 1.0000 F1 Score: 1.0000 Spearman Corr: 1.0000 Specificity: 1.0000 QWK: 1.0000 Mean Metrics: 1.0000


Validation: 100%|██████████| 99/99 [00:00<00:00, 1249.03it/s]

Val Loss: 2.5471 Val Acc: 0.7475
Validation Accuracy: 0.7475 F1 Score: 0.7276 Spearman Corr: -0.1996 Specificity: 0.7148 QWK: -0.1967 Mean Metrics: 0.2615


1-1 again

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os


def extract_and_save_features(dataset, feature_extractor, save_path):
    feature_extractor.eval()  # 确保模型处于评估模式
    all_features = []
    all_labels = []
    
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        features = sample['features']
        label = sample['label']
        
        all_features.append(features)
        all_labels.append(label)
    
    all_features = torch.stack(all_features)
    all_labels = torch.tensor(all_labels)
    
    torch.save({'features': all_features, 'labels': all_labels}, save_path)
    print(f"Features and labels saved to {save_path}")


# 忽略所有警告
warnings.filterwarnings("ignore")
# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Total number of localizers in dataset: 394
Total number of batches: 394
Epoch 0/300
----------


Training: 100%|██████████| 394/394 [00:00<00:00, 629.17it/s]


Loss: 0.6020 Acc: 0.8173
Train Accuracy: 0.8173 F1 Score: 0.7562 Spearman Corr: 0.2210 Specificity: 0.6900 QWK: 0.1431 Mean Metrics: 0.4526


Validation: 100%|██████████| 99/99 [00:00<00:00, 1703.29it/s]


Val Loss: 0.5972 Val Acc: 0.8182
Validation Accuracy: 0.8182 F1 Score: 0.7555 Spearman Corr: -0.0123 Specificity: 0.6961 QWK: -0.0117 Mean Metrics: 0.3569
Epoch 1/300
----------


...

Val Loss: 2.7578 Val Acc: 0.7172
Validation Accuracy: 0.7172 F1 Score: 0.7170 Spearman Corr: -0.1177 Specificity: 0.7183 QWK: -0.1161 Mean Metrics: 0.3004
Epoch 300/300
----------


Training: 100%|██████████| 394/394 [00:00<00:00, 759.57it/s]


Loss: 0.0180 Acc: 0.9924
Train Accuracy: 0.9924 F1 Score: 0.9923 Spearman Corr: 0.9769 Specificity: 0.9891 QWK: 0.9774 Mean Metrics: 0.9839


Validation: 100%|██████████| 99/99 [00:00<00:00, 1791.19it/s]

Val Loss: 2.8040 Val Acc: 0.6768
Validation Accuracy: 0.6768 F1 Score: 0.6891 Spearman Corr: 0.0425 Specificity: 0.6886 QWK: 0.0461 Mean Metrics: 0.3666


In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os


def extract_and_save_features(dataset, feature_extractor, save_path):
    feature_extractor.eval()  # 确保模型处于评估模式
    all_features = []
    all_labels = []
    
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        features = sample['features']
        label = sample['label']
        
        all_features.append(features)
        all_labels.append(label)
    
    all_features = torch.stack(all_features)
    all_labels = torch.tensor(all_labels)
    
    torch.save({'features': all_features, 'labels': all_labels}, save_path)
    print(f"Features and labels saved to {save_path}")


# 忽略所有警告
warnings.filterwarnings("ignore")
# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9735 images.
Processed 101/9735 images.
Processed 201/9735 images.
Processed 301/9735 images.
Processed 401/9735 images.
Processed 501/9735 images.
Processed 601/9735 images.
Processed 701/9735 images.
Processed 801/9735 images.
Processed 901/9735 images.
Processed 1001/9735 images.
Processed 1101/9735 images.
Processed 1201/9735 images.
Processed 1301/9735 images.
Processed 1401/9735 images.
Processed 1501/9735 images.
Processed 1601/9735 images.
Processed 1701/9735 images.
Processed 1801/9735 images.
Processed 1901/9735 images.
Processed 2001/9735 images.
Processed 2101/9735 images.
Processed 2201/9735 images.
Processed 2301/9735 images.
Processed 2401/9735 images.
Processed 2501/9735 images.
Processed 2601/9735 images.
Processed 2701/9735 images.
Processed 2801/9735 images.
Processed 2901/9735 images.
Processed 3001/9735 images.
Processed 3101/9735 images.
Processed 3201/9735 images.
Processed 3301/97

Training: 100%|██████████| 394/394 [00:00<00:00, 718.34it/s]


Loss: 0.7780 Acc: 0.7690
Train Accuracy: 0.7690 F1 Score: 0.6730 Spearman Corr: 0.0000 Specificity: 0.6648 QWK: 0.0000 Mean Metrics: 0.3345


Validation: 100%|██████████| 99/99 [00:00<00:00, 1578.39it/s]


Val Loss: 0.7028 Val Acc: 0.8081
Validation Accuracy: 0.8081 F1 Score: 0.7223 Spearman Corr: nan Specificity: 0.6667 QWK: 0.0000 Mean Metrics: nan
Epoch 1/300
----------


...

Val Loss: 2.4096 Val Acc: 0.7475
Validation Accuracy: 0.7475 F1 Score: 0.7418 Spearman Corr: 0.2145 Specificity: 0.7270 QWK: 0.2156 Mean Metrics: 0.4747
Epoch 300/300
----------


Training: 100%|██████████| 394/394 [00:00<00:00, 928.57it/s]


Loss: 0.0011 Acc: 1.0000
Train Accuracy: 1.0000 F1 Score: 1.0000 Spearman Corr: 1.0000 Specificity: 1.0000 QWK: 1.0000 Mean Metrics: 1.0000


Validation: 100%|██████████| 99/99 [00:00<00:00, 1848.55it/s]

Val Loss: 2.4057 Val Acc: 0.7273
Validation Accuracy: 0.7273 F1 Score: 0.7304 Spearman Corr: 0.1436 Specificity: 0.7333 QWK: 0.1459 Mean Metrics: 0.4383


14

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os


def extract_and_save_features(dataset, feature_extractor, save_path):
    feature_extractor.eval()  # 确保模型处于评估模式
    all_features = []
    all_labels = []
    
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        features = sample['features']
        label = sample['label']
        
        all_features.append(features)
        all_labels.append(label)
    
    all_features = torch.stack(all_features)
    all_labels = torch.tensor(all_labels)
    
    torch.save({'features': all_features, 'labels': all_labels}, save_path)
    print(f"Features and labels saved to {save_path}")


# 忽略所有警告
warnings.filterwarnings("ignore")
# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9418 images.
Processed 101/9418 images.
Processed 201/9418 images.
Processed 301/9418 images.
Processed 401/9418 images.
Processed 501/9418 images.
Processed 601/9418 images.
Processed 701/9418 images.
Processed 801/9418 images.
Processed 901/9418 images.
Processed 1001/9418 images.
Processed 1101/9418 images.
Processed 1201/9418 images.
Processed 1301/9418 images.
Processed 1401/9418 images.
Processed 1501/9418 images.
Processed 1601/9418 images.
Processed 1701/9418 images.
Processed 1801/9418 images.
Processed 1901/9418 images.
Processed 2001/9418 images.
Processed 2101/9418 images.
Processed 2201/9418 images.
Processed 2301/9418 images.
Processed 2401/9418 images.
Processed 2501/9418 images.
Processed 2601/9418 images.
Processed 2701/9418 images.
Processed 2801/9418 images.
Processed 2901/9418 images.
Processed 3001/9418 images.
Processed 3101/9418 images.
Processed 3201/9418 images.
Processed 3301/94

Training: 100%|██████████| 395/395 [00:00<00:00, 692.73it/s]


Loss: 0.6499 Acc: 0.8228
Train Accuracy: 0.8228 F1 Score: 0.7570 Spearman Corr: 0.0021 Specificity: 0.6713 QWK: 0.0014 Mean Metrics: 0.3579


Validation: 100%|██████████| 98/98 [00:00<00:00, 1678.24it/s]


Val Loss: 0.7762 Val Acc: 0.7551
Validation Accuracy: 0.7551 F1 Score: 0.6497 Spearman Corr: nan Specificity: 0.6667 QWK: 0.0000 Mean Metrics: nan
Epoch 1/300
----------


...

Val Loss: 1.7866 Val Acc: 0.7041
Validation Accuracy: 0.7041 F1 Score: 0.6917 Spearman Corr: 0.1312 Specificity: 0.7372 QWK: 0.1315 Mean Metrics: 0.4229
Epoch 300/300
----------


Training: 100%|██████████| 395/395 [00:00<00:00, 785.41it/s]


Loss: 0.0941 Acc: 0.9696
Train Accuracy: 0.9696 F1 Score: 0.9691 Spearman Corr: 0.9127 Specificity: 0.9582 QWK: 0.9110 Mean Metrics: 0.9378


Validation: 100%|██████████| 98/98 [00:00<00:00, 1799.83it/s]

Val Loss: 1.9337 Val Acc: 0.7347
Validation Accuracy: 0.7347 F1 Score: 0.7178 Spearman Corr: 0.2917 Specificity: 0.7388 QWK: 0.2906 Mean Metrics: 0.5097


In [ ]:
1-5

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os


def extract_and_save_features(dataset, feature_extractor, save_path):
    feature_extractor.eval()  # 确保模型处于评估模式
    all_features = []
    all_labels = []
    
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        features = sample['features']
        label = sample['label']
        
        all_features.append(features)
        all_labels.append(label)
    
    all_features = torch.stack(all_features)
    all_labels = torch.tensor(all_labels)
    
    torch.save({'features': all_features, 'labels': all_labels}, save_path)
    print(f"Features and labels saved to {save_path}")


# 忽略所有警告
warnings.filterwarnings("ignore")
# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9394 images.
Processed 101/9394 images.
Processed 201/9394 images.
Processed 301/9394 images.
Processed 401/9394 images.
Processed 501/9394 images.
Processed 601/9394 images.
Processed 701/9394 images.
Processed 801/9394 images.
Processed 901/9394 images.
Processed 1001/9394 images.
Processed 1101/9394 images.
Processed 1201/9394 images.
Processed 1301/9394 images.
Processed 1401/9394 images.
Processed 1501/9394 images.
Processed 1601/9394 images.
Processed 1701/9394 images.
Processed 1801/9394 images.
Processed 1901/9394 images.
Processed 2001/9394 images.
Processed 2101/9394 images.
Processed 2201/9394 images.
Processed 2301/9394 images.
Processed 2401/9394 images.
Processed 2501/9394 images.
Processed 2601/9394 images.
Processed 2701/9394 images.
Processed 2801/9394 images.
Processed 2901/9394 images.
Processed 3001/9394 images.
Processed 3101/9394 images.
Processed 3201/9394 images.
Processed 3301/93

Training: 100%|██████████| 395/395 [00:00<00:00, 682.91it/s]


Loss: 0.6629 Acc: 0.7949
Train Accuracy: 0.7949 F1 Score: 0.7233 Spearman Corr: 0.1399 Specificity: 0.6786 QWK: 0.0815 Mean Metrics: 0.4058


Validation: 100%|██████████| 98/98 [00:00<00:00, 1784.97it/s]


Val Loss: 0.7490 Val Acc: 0.7857
Validation Accuracy: 0.7857 F1 Score: 0.6954 Spearman Corr: -0.2156 Specificity: 0.6788 QWK: -0.0879 Mean Metrics: 0.2677
Epoch 1/300
----------


...

Val Loss: 2.9243 Val Acc: 0.6531
Validation Accuracy: 0.6531 F1 Score: 0.6391 Spearman Corr: 0.1078 Specificity: 0.6413 QWK: 0.1067 Mean Metrics: 0.3737
Epoch 300/300
----------


Training: 100%|██████████| 395/395 [00:00<00:00, 797.97it/s]


Loss: 0.0028 Acc: 1.0000
Train Accuracy: 1.0000 F1 Score: 1.0000 Spearman Corr: 1.0000 Specificity: 1.0000 QWK: 1.0000 Mean Metrics: 1.0000


Validation: 100%|██████████| 98/98 [00:00<00:00, 1807.13it/s]

Val Loss: 3.1910 Val Acc: 0.6837
Validation Accuracy: 0.6837 F1 Score: 0.6492 Spearman Corr: 0.0602 Specificity: 0.6406 QWK: 0.0571 Mean Metrics: 0.3518


1-1 new

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os
import random



# 忽略所有警告
warnings.filterwarnings("ignore")


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # 确保 CuDNN 使用确定性算法
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1) 

# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9495 images.
Processed 101/9495 images.
Processed 201/9495 images.
Processed 301/9495 images.
Processed 401/9495 images.
Processed 501/9495 images.
Processed 601/9495 images.
Processed 701/9495 images.
Processed 801/9495 images.
Processed 901/9495 images.
Processed 1001/9495 images.
Processed 1101/9495 images.
Processed 1201/9495 images.
Processed 1301/9495 images.
Processed 1401/9495 images.
Processed 1501/9495 images.
Processed 1601/9495 images.
Processed 1701/9495 images.
Processed 1801/9495 images.
Processed 1901/9495 images.
Processed 2001/9495 images.
Processed 2101/9495 images.
Processed 2201/9495 images.
Processed 2301/9495 images.
Processed 2401/9495 images.
Processed 2501/9495 images.
Processed 2601/9495 images.
Processed 2701/9495 images.
Processed 2801/9495 images.
Processed 2901/9495 images.
Processed 3001/9495 images.
Processed 3101/9495 images.
Processed 3201/9495 images.
Processed 3301/94

Training: 100%|██████████| 394/394 [00:01<00:00, 284.78it/s]


Loss: 0.6512 Acc: 0.8147
Train Accuracy: 0.8147 F1 Score: 0.7424 Spearman Corr: 0.1002 Specificity: 0.6731 QWK: 0.0488 Mean Metrics: 0.3911


Validation: 100%|██████████| 99/99 [00:00<00:00, 833.39it/s]


Val Loss: 0.6944 Val Acc: 0.8182
Validation Accuracy: 0.8182 F1 Score: 0.7525 Spearman Corr: 0.1572 Specificity: 0.6814 QWK: 0.0927 Mean Metrics: 0.4209
Epoch 1/300
----------


...

Val Loss: 3.6751 Val Acc: 0.7374
Validation Accuracy: 0.7374 F1 Score: 0.7313 Spearman Corr: 0.1260 Specificity: 0.6966 QWK: 0.1234 Mean Metrics: 0.4193
Epoch 300/300
----------


Training: 100%|██████████| 394/394 [00:01<00:00, 286.43it/s]


Loss: 0.0435 Acc: 0.9924
Train Accuracy: 0.9924 F1 Score: 0.9923 Spearman Corr: 0.9803 Specificity: 0.9868 QWK: 0.9798 Mean Metrics: 0.9848


Validation: 100%|██████████| 99/99 [00:00<00:00, 778.04it/s]

Val Loss: 3.3110 Val Acc: 0.6566
Validation Accuracy: 0.6566 F1 Score: 0.6837 Spearman Corr: 0.1122 Specificity: 0.6972 QWK: 0.1105 Mean Metrics: 0.4009


1-2new

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os
import random



# 忽略所有警告
warnings.filterwarnings("ignore")


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # 确保 CuDNN 使用确定性算法
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1) 

# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9574 images.
Processed 101/9574 images.
Processed 201/9574 images.
Processed 301/9574 images.
Processed 401/9574 images.
Processed 501/9574 images.
Processed 601/9574 images.
Processed 701/9574 images.
Processed 801/9574 images.
Processed 901/9574 images.
Processed 1001/9574 images.
Processed 1101/9574 images.
Processed 1201/9574 images.
Processed 1301/9574 images.
Processed 1401/9574 images.
Processed 1501/9574 images.
Processed 1601/9574 images.
Processed 1701/9574 images.
Processed 1801/9574 images.
Processed 1901/9574 images.
Processed 2001/9574 images.
Processed 2101/9574 images.
Processed 2201/9574 images.
Processed 2301/9574 images.
Processed 2401/9574 images.
Processed 2501/9574 images.
Processed 2601/9574 images.
Processed 2701/9574 images.
Processed 2801/9574 images.
Processed 2901/9574 images.
Processed 3001/9574 images.
Processed 3101/9574 images.
Processed 3201/9574 images.
Processed 3301/95

Training: 100%|██████████| 394/394 [00:01<00:00, 306.31it/s]


Loss: 0.7639 Acc: 0.7690
Train Accuracy: 0.7690 F1 Score: 0.6860 Spearman Corr: -0.0384 Specificity: 0.6707 QWK: -0.0224 Mean Metrics: 0.3240


Validation: 100%|██████████| 99/99 [00:00<00:00, 885.88it/s]


Val Loss: 0.6506 Val Acc: 0.7879
Validation Accuracy: 0.7879 F1 Score: 0.6983 Spearman Corr: -0.2132 Specificity: 0.6789 QWK: -0.0859 Mean Metrics: 0.2695
Epoch 1/300
----------


...

Val Loss: 2.1882 Val Acc: 0.7071
Validation Accuracy: 0.7071 F1 Score: 0.7018 Spearman Corr: 0.1516 Specificity: 0.7213 QWK: 0.1505 Mean Metrics: 0.4313
Epoch 300/300
----------


Training: 100%|██████████| 394/394 [00:01<00:00, 335.89it/s]


Loss: 0.0208 Acc: 0.9924
Train Accuracy: 0.9924 F1 Score: 0.9923 Spearman Corr: 0.9795 Specificity: 0.9897 QWK: 0.9786 Mean Metrics: 0.9850


Validation: 100%|██████████| 99/99 [00:00<00:00, 1053.04it/s]

Val Loss: 2.4062 Val Acc: 0.6869
Validation Accuracy: 0.6869 F1 Score: 0.6917 Spearman Corr: 0.1201 Specificity: 0.7269 QWK: 0.1133 Mean Metrics: 0.4130


1-3

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os
import random



# 忽略所有警告
warnings.filterwarnings("ignore")


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # 确保 CuDNN 使用确定性算法
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1) 

# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9735 images.
Processed 101/9735 images.
Processed 201/9735 images.
Processed 301/9735 images.
Processed 401/9735 images.
Processed 501/9735 images.
Processed 601/9735 images.
Processed 701/9735 images.
Processed 801/9735 images.
Processed 901/9735 images.
Processed 1001/9735 images.
Processed 1101/9735 images.
Processed 1201/9735 images.
Processed 1301/9735 images.
Processed 1401/9735 images.
Processed 1501/9735 images.
Processed 1601/9735 images.
Processed 1701/9735 images.
Processed 1801/9735 images.
Processed 1901/9735 images.
Processed 2001/9735 images.
Processed 2101/9735 images.
Processed 2201/9735 images.
Processed 2301/9735 images.
Processed 2401/9735 images.
Processed 2501/9735 images.
Processed 2601/9735 images.
Processed 2701/9735 images.
Processed 2801/9735 images.
Processed 2901/9735 images.
Processed 3001/9735 images.
Processed 3101/9735 images.
Processed 3201/9735 images.
Processed 3301/97

Training: 100%|██████████| 394/394 [00:01<00:00, 351.87it/s]


Loss: 0.8158 Acc: 0.7563
Train Accuracy: 0.7563 F1 Score: 0.6668 Spearman Corr: 0.1536 Specificity: 0.6775 QWK: 0.0772 Mean Metrics: 0.3938


Validation: 100%|██████████| 99/99 [00:00<00:00, 1049.68it/s]


Val Loss: 0.6456 Val Acc: 0.7980
Validation Accuracy: 0.7980 F1 Score: 0.7173 Spearman Corr: 0.0077 Specificity: 0.6630 QWK: 0.0030 Mean Metrics: 0.3478
Epoch 1/300
----------


...

Val Loss: 2.0977 Val Acc: 0.7576
Validation Accuracy: 0.7576 F1 Score: 0.7251 Spearman Corr: 0.0621 Specificity: 0.6892 QWK: 0.0611 Mean Metrics: 0.3844
Epoch 300/300
----------


Training: 100%|██████████| 394/394 [00:01<00:00, 336.04it/s]


Loss: 0.0624 Acc: 0.9797
Train Accuracy: 0.9797 F1 Score: 0.9797 Spearman Corr: 0.9550 Specificity: 0.9816 QWK: 0.9563 Mean Metrics: 0.9682


Validation: 100%|██████████| 99/99 [00:00<00:00, 898.33it/s]

Val Loss: 2.4737 Val Acc: 0.7576
Validation Accuracy: 0.7576 F1 Score: 0.7155 Spearman Corr: 0.1679 Specificity: 0.6759 QWK: 0.1520 Mean Metrics: 0.4278


11-4

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os
import random



# 忽略所有警告
warnings.filterwarnings("ignore")


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # 确保 CuDNN 使用确定性算法
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1) 

# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9418 images.
Processed 101/9418 images.
Processed 201/9418 images.
Processed 301/9418 images.
Processed 401/9418 images.
Processed 501/9418 images.
Processed 601/9418 images.
Processed 701/9418 images.
Processed 801/9418 images.
Processed 901/9418 images.
Processed 1001/9418 images.
Processed 1101/9418 images.
Processed 1201/9418 images.
Processed 1301/9418 images.
Processed 1401/9418 images.
Processed 1501/9418 images.
Processed 1601/9418 images.
Processed 1701/9418 images.
Processed 1801/9418 images.
Processed 1901/9418 images.
Processed 2001/9418 images.
Processed 2101/9418 images.
Processed 2201/9418 images.
Processed 2301/9418 images.
Processed 2401/9418 images.
Processed 2501/9418 images.
Processed 2601/9418 images.
Processed 2701/9418 images.
Processed 2801/9418 images.
Processed 2901/9418 images.
Processed 3001/9418 images.
Processed 3101/9418 images.
Processed 3201/9418 images.
Processed 3301/94

Training: 100%|██████████| 395/395 [00:01<00:00, 326.87it/s]


Loss: 0.6964 Acc: 0.7747
Train Accuracy: 0.7747 F1 Score: 0.7097 Spearman Corr: 0.1182 Specificity: 0.6835 QWK: 0.0886 Mean Metrics: 0.4000


Validation: 100%|██████████| 98/98 [00:00<00:00, 917.63it/s]


Val Loss: 0.7793 Val Acc: 0.7653
Validation Accuracy: 0.7653 F1 Score: 0.6722 Spearman Corr: 0.2076 Specificity: 0.6806 QWK: 0.0815 Mean Metrics: 0.4105
Epoch 1/300
----------


...

Val Loss: 3.6856 Val Acc: 0.7449
Validation Accuracy: 0.7449 F1 Score: 0.6828 Spearman Corr: 0.2380 Specificity: 0.6928 QWK: 0.1967 Mean Metrics: 0.4526
Epoch 300/300
----------


Training: 100%|██████████| 395/395 [00:01<00:00, 348.50it/s]


Loss: 0.1284 Acc: 0.9747
Train Accuracy: 0.9747 F1 Score: 0.9745 Spearman Corr: 0.8841 Specificity: 0.9716 QWK: 0.8840 Mean Metrics: 0.9285


Validation: 100%|██████████| 98/98 [00:00<00:00, 1063.78it/s]

Val Loss: 2.9180 Val Acc: 0.7041
Validation Accuracy: 0.7041 F1 Score: 0.6796 Spearman Corr: 0.2081 Specificity: 0.7075 QWK: 0.2033 Mean Metrics: 0.4496


1-5

In [1]:
from PIL import Image
from tqdm import tqdm

from sklearn.metrics import cohen_kappa_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import pandas as pd
from config import config
from OCTData import MILDataset, TestData
from models import ConvNeXtMILModel, CLAMResNet50, resnet50_feature_extractor,convnext2_feature_extractor
from train_mil import train_abmil_model
from evaluate import evaluate_model
import numpy as np
from torch import nn, optim
import warnings
from OCTData import LocalizerGroupedDataset
from MIL_models import ABMIL, ABMIL_MultiHead, CLAM_SB
from resnet_custom import resnet50_baseline
import os
import random



# 忽略所有警告
warnings.filterwarnings("ignore")


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # 确保 CuDNN 使用确定性算法
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(1) 

# Define transformations
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=config.normalize_mean, std=config.normalize_std)
])

# train_transform = transforms.Compose([
#         transforms.RandomApply([transforms.ColorJitter(brightness=0.25, contrast=0.25)], p=0.25),
#         transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.15),
#         transforms.RandomAffine(degrees=25, scale=(0.8, 1.2), translate=(0.1, 0.1)),
#         transforms.RandomHorizontalFlip(p=0.3),
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.5], std=[0.5]),
#     ])

# Load and check datasets
data_frame = pd.read_csv(config.train_csv_file)

data_frame = data_frame[['label', 'id_patient', 'image','case']]

# 打印列名进行检查
print("数据框列名:", data_frame.columns)

# 确保数据框中有'label'列
if 'label' not in data_frame.columns:
    raise ValueError("'label' 列在数据框中不存在，请检查CSV文件中的列名。")

train_df= pd.read_csv(config.final_train_csv_file)
val_df = pd.read_csv(config.final_val_csv_file)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = convnext2_feature_extractor().to(device)



def extract_and_save_features_with_labels(data_frame, root_dir, transform, feature_extractor, save_path):
    features_dict = {}
    labels_dict = {}
    total_images = len(data_frame)
    for idx, row in data_frame.iterrows():
        img_name = os.path.join(root_dir, row['image'])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        if transform:
            image = transform(image).to(device) 
        with torch.no_grad():
            feature = feature_extractor(image.unsqueeze(0))
        localizer_key = row['LOCALIZER']
        if localizer_key not in features_dict:
            features_dict[localizer_key] = []
            labels_dict[localizer_key] = int(row['label'])  # Assign label only once
        features_dict[localizer_key].append(feature.squeeze(0))
        if idx % 100 == 0 or idx == total_images - 1:
            print(f"Processed {idx + 1}/{total_images} images.")
    # Save the features and labels to the specified path
    torch.save({'features': features_dict, 'labels': labels_dict}, save_path)
    
    print(f"Features and labels saved to {save_path}")

def load_features_and_labels_from_file(save_path):
    saved_data = torch.load(save_path)
    return saved_data['features'], saved_data['labels']

# Extract features and labels, and save them
train_features_path = config.feature_train_save_path
if not os.path.exists(train_features_path):
    extract_and_save_features_with_labels(train_df, config.final_root_dir, transform, feature_extractor, train_features_path)

val_features_path = config.feature_val_save_path
if not os.path.exists(val_features_path):
    extract_and_save_features_with_labels(val_df, config.final_root_dir, transform, feature_extractor, val_features_path)

train_features, train_labels = load_features_and_labels_from_file(train_features_path)

train_dataset = LocalizerGroupedDataset(features_dict=train_features, labels_dict=train_labels)

val_features, val_labels = load_features_and_labels_from_file(val_features_path)

val_dataset = LocalizerGroupedDataset(features_dict=val_features, labels_dict=val_labels)

# # Prepare datasets
# train_dataset = LocalizerGroupedDataset(data_frame=train_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)
# val_dataset = LocalizerGroupedDataset(data_frame=val_df, root_dir=config.final_root_dir, transform=transform, feature_extractor=feature_extractor)



# Initialize model, criterion, optimizer
# model = ABMIL().to(config.device)
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
# model = ABMIL_MultiHead(n_classes=3, num_heads=2, dropout=0.25).to('cuda')
model = CLAM_SB(gate=True, size_arg="small", n_classes=3, embed_dim=1000).to(config.device)


localizer_labels = train_df.groupby('LOCALIZER')['label'].apply(lambda x: x.mode()[0])

# Calculate sample weights for each localizer based on the most common label
class_counts = localizer_labels.value_counts().to_dict()
localizer_sample_weights = localizer_labels.map(lambda x: 1.0 / class_counts[x])
sampler = WeightedRandomSampler(weights=localizer_sample_weights.values, num_samples=len(localizer_sample_weights), replacement=True)



# 在DataLoader中使用采样器
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=sampler, drop_last=True)
# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=config.batch_size,drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False,drop_last=True)

print(f"Total number of localizers in dataset: {len(train_loader.dataset)}")
print(f"Total number of batches: {len(train_loader)}")


criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=config.lr)

# Train the model
trained_model = train_abmil_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=config.num_epochs)



数据框列名: Index(['label', 'id_patient', 'image', 'case'], dtype='object')
Processed 1/9394 images.
Processed 101/9394 images.
Processed 201/9394 images.
Processed 301/9394 images.
Processed 401/9394 images.
Processed 501/9394 images.
Processed 601/9394 images.
Processed 701/9394 images.
Processed 801/9394 images.
Processed 901/9394 images.
Processed 1001/9394 images.
Processed 1101/9394 images.
Processed 1201/9394 images.
Processed 1301/9394 images.
Processed 1401/9394 images.
Processed 1501/9394 images.
Processed 1601/9394 images.
Processed 1701/9394 images.
Processed 1801/9394 images.
Processed 1901/9394 images.
Processed 2001/9394 images.
Processed 2101/9394 images.
Processed 2201/9394 images.
Processed 2301/9394 images.
Processed 2401/9394 images.
Processed 2501/9394 images.
Processed 2601/9394 images.
Processed 2701/9394 images.
Processed 2801/9394 images.
Processed 2901/9394 images.
Processed 3001/9394 images.
Processed 3101/9394 images.
Processed 3201/9394 images.
Processed 3301/93

Training: 100%|██████████| 395/395 [00:01<00:00, 327.72it/s]


Loss: 0.6977 Acc: 0.8203
Train Accuracy: 0.8203 F1 Score: 0.7393 Spearman Corr: nan Specificity: 0.6667 QWK: 0.0000 Mean Metrics: nan


Validation: 100%|██████████| 98/98 [00:00<00:00, 811.96it/s]


Val Loss: 0.7693 Val Acc: 0.7959
Validation Accuracy: 0.7959 F1 Score: 0.7161 Spearman Corr: 0.0053 Specificity: 0.6947 QWK: 0.0053 Mean Metrics: 0.3554
Epoch 1/300
----------


...

Val Loss: 2.8558 Val Acc: 0.7347
Validation Accuracy: 0.7347 F1 Score: 0.7155 Spearman Corr: 0.1151 Specificity: 0.7202 QWK: 0.1156 Mean Metrics: 0.4166
Epoch 300/300
----------


Training: 100%|██████████| 395/395 [00:01<00:00, 393.30it/s]


Loss: 0.0420 Acc: 0.9924
Train Accuracy: 0.9924 F1 Score: 0.9924 Spearman Corr: 0.9809 Specificity: 0.9937 QWK: 0.9799 Mean Metrics: 0.9867


Validation: 100%|██████████| 98/98 [00:00<00:00, 1093.04it/s]

Val Loss: 2.8686 Val Acc: 0.6633
Validation Accuracy: 0.6633 F1 Score: 0.6664 Spearman Corr: -0.0578 Specificity: 0.7049 QWK: -0.0586 Mean Metrics: 0.3137
